In [ ]:
%reload_ext autoreload
%autoreload 2
%autosave 120
%matplotlib inline
import numpy as np, matplotlib.pyplot as plt
from os.path import join

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Step 1. Validate the data
    use the example in the cell below, to verify the TFL patch you've generated is sane.
    Things to watch for:
    1. You are able to load and vizualize your train and val data, using the functions below.
    2. using the vizualization verify  image <--> label correspondence is correct.
    3. % Negative vs. Positive examples is aprroximately 50%
 

In [ ]:
def load_tfl_data(data_dir, crop_shape=(81,81)):
    images = np.memmap(join(data_dir,'data.bin'),mode='r',dtype=np.uint8).reshape([-1]+list(crop_shape) +[3])
    labels = np.memmap(join(data_dir,'labels.bin'),mode='r',dtype=np.uint8)
    return {'images':images,'labels':labels}

def viz_my_data(images,labels, predictions=None, num=(5,5), labels2name= {0:'No TFL',1:'Yes TFL'}):
    assert images.shape[0] == labels.shape[0]
    assert predictions is None or predictions.shape[0] == images.shape[0]
    h = 5
    n = num[0]*num[1]
    ax = plt.subplots(num[0],num[1],figsize=(h*num[0],h*num[1]),gridspec_kw={'wspace':0.05},squeeze=False,sharex=True,sharey=True)[1]#.flatten()
    idxs = np.random.randint(0,images.shape[0],n)
    for i,idx in enumerate(idxs):
        ax.flatten()[i].imshow(images[idx])
        title = labels2name[labels[idx]]
        if predictions is not None : title += ' Prediction: {:.2f}'.format(predictions[idx])
        ax.flatten()[i].set_title(title)
   
data_dir = "/content/drive/MyDrive/Colab Notebooks/data"
datasets = {
    'val':load_tfl_data(join(data_dir,'val')),
    'train': load_tfl_data(join(data_dir,'train')),
    }
for k,v in datasets.items():
    
    print ('{} :  {} 0/1 split {:.1f} %'.format(k,v['images'].shape, np.mean(v['labels']==1)*100))

viz_my_data(num=(6,6),**datasets['val'])   

## define the model used for training


In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.models import Model
import keras
restnet = ResNet50(include_top=False, weights='imagenet', input_shape=(81,81,3))
output = restnet.layers[-1].output
output = keras.layers.Flatten()(output)
restnet = Model(restnet.input, outputs=output)
for layer in restnet.layers:
    layer.trainable = False
restnet.summary()

In [ ]:
[n,w,h,c] = [5,5,5,3]
tf.compat.v1.disable_eager_execution()
ph = tf.compat.v1.placeholder(shape=[None,w,h,c],dtype=tf.float32)
model = dropblock(ph,0.1,3)
out = sess.run(model,feed_dict={ph:input})
print(out[0,:,:,0])

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras import regularizers
model = Sequential()
model.add(restnet)
model.add(Dense(128, activation='relu', input_dim=(81,81,3)))
model.add(Dropout(0.3))
model.add(Dense(128,activation=('relu'),input_dim=(81,81,3)))
model.add(Dropout(0.3))
model.add(Dense(2, activation='sigmoid'))
model.summary()


## train

In [ ]:
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.optimizers.schedules import InverseTimeDecay
data_dir = '/content/drive/MyDrive/Colab Notebooks/data'
datasets = {
    'val':load_tfl_data(join(data_dir,'val')),
    'train': load_tfl_data(join(data_dir,'train')),
    }
N_VALIDATION = int(1e3)
N_TRAIN = int(1e4)
BUFFER_SIZE = int(1e4)
BATCH_SIZE = 500
STEPS_PER_EPOCH = N_TRAIN//BATCH_SIZE
lr_schedule = InverseTimeDecay(
  0.001,
  decay_steps=STEPS_PER_EPOCH*1000,
  decay_rate=1,
  staircase=False)
model.compile(optimizer=Adam(),loss =sparse_categorical_crossentropy,metrics=['accuracy'])
train,val = datasets['train'],datasets['val']
#train it, the model uses the 'train' dataset for learning. We evaluate the "goodness" of the model, by predicting the label of the images in the val dataset.
history=model.fit(train['images'],train['labels'],validation_data=(val['images'],val['labels']),epochs =4)

In [ ]:
epochs = history.history
epochs['train_accuracy'] = epochs['accuracy']
plt.figure(figsize=(10,10))
for k in ['train_accuracy','val_accuracy']:
    plt.plot(range(len(epochs[k])), epochs[k],label=k)

plt.legend();

In [ ]:
epochs = history.history
epochs['train_loss'] = epochs['loss']
plt.figure(figsize=(10,10))
for k in ['train_loss','val_loss']:
    plt.plot(range(len(epochs[k])), epochs[k],label=k)

plt.legend();

## evaluate and predict


In [ ]:
import seaborn as sbn
predictions = model.predict(val['images'])
sbn.distplot(predictions[:,0]);

predicted_label = np.argmax(predictions, axis=-1)
print ('accuracy:', np.mean(predicted_label==val['labels']))

In [ ]:
viz_my_data(num=(6,6),predictions=predictions[:,1],**val);

### Saving the model
After we trained our model and made predictions with it, we will now want to save the **architecture** together with its learned **weights** in order for us to be able to use it in the TFL manager.
The architecture will be saved as a json, the weights in the h5 format: 

In [ ]:
model.save("/content/drive/MyDrive/Colab Notebooks/model.h5")